In [1]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

# For importing .xml files
import xml.etree.ElementTree as ET

# For handeling dataframes
import pandas as pd
import numpy as np
import re                                  # library for regular expression operations
import string                              # for string operations

# For text preprocessing
import nltk                                # Natural Language Toolkit
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer as ps  # module for stemming

# For building neural netwrok models
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import transformers

# For model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score
import time


In [2]:
# Load the data
labels_all = pd.read_csv("data/labels_all.csv")
labels_all.drop(['Unnamed: 0'], axis=1, inplace=True)

text_all = pd.read_csv('data/text_all.csv')
text_all.drop(['Unnamed: 0'], axis=1, inplace=True)

#  Create binary labels for id
disease_types = ['Hypertriglyceridemia','Venous Insufficiency','Asthma','Gout','OSA','PVD','Gallstones','OA','GERD',
                'Depression','Obesity','CHF','Hypercholesterolemia','CAD','Diabetes','Hypertension']

In [3]:
def make_binary(labels_df):
    for i in range(16):
        # In the disease column, code the disease of interest as 1 and other disease types as 0 to indicate 
        # whether this row is about this particular disease
        labels_df["disease_"+str(i)] = [1 if x==disease_types[i] else 0 for x in labels_df["disease"]]
        # In the label column, code Y as 1 and everything else as 0 to indicate whether a disease is present
        # (it may or may not be about this particular disease)
        labels_df["label_"+str(i)] = [1 if x=="Y" else 0 for x in labels_df["label"]] 
        # Create a "This_Disease" column that indicates whether a patient has this particular disease or not
        # A patient is coded as having this disease only when (disease is 1) AND (label is 1)
        labels_df["This_Disease_"+str(i)] = labels_df["disease_"+str(i)] * labels_df["label_"+str(i)] 
        labels_df.drop(["disease_"+str(i), "label_"+str(i)], axis=1, inplace=True)

    # Now that all useful info in "disease" and "Label" are combined in column "This_Disease", no longer need disease and label
    labels_df.drop(['label', 'disease'], axis=1, inplace=True)
    # Duplicates after removing disease and label are due to 0's in the "disease" column if the patient had other disease info
    labels_df.drop_duplicates(inplace = True)
    # Aggregate the binary values into one row for each id
    labels_df = labels_df.groupby('id').aggregate('sum').reset_index()
    
    return(labels_df)

In [4]:
binary_labels = make_binary(labels_all)

# Calculate the disease prevalence for each disease type
prevalence = binary_labels.aggregate('sum')/1237


# Combine the x_test and y_test sets
df = text_all.merge(binary_labels, on="id", how="left")

In [5]:
# Download the stopwords from NLTK
nltk.download('stopwords')

# Import the standard English stop words list from NLTK
stopwords_english = stopwords.words('english') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
additional_sw = ['admission', 'date', 'discharge', 'summary', 'diagnoses', 'diagnosis', 'history',
                    'patient', 'report', 'status', 'signed', 'attending', 'service', 'dictated', 'dictation']
all_stopwords = additional_sw + stopwords_english

In [7]:
# # Convert all words in text column to lower case in both the train and test sets
# df['text'] = [str(x).lower() for x in df['text']]

# # Remove noiseb
# df['text']  = [re.sub("(\W|\d+|\n)", " ", elem).strip() for elem in df['text']] # remove spaces, digits and line breaks

# # Remove punctuations
# def remove_punctuations(text):
#     return ' '.join(['' if (elem in string.punctuation) else elem for elem in text.split()])

# df['text'] = [remove_punctuations(elem) for elem in df['text']]


# # Remove standard English stop words
# def remove_stopwords(text):
#     return ' '.join(['' if (elem in all_stopwords) else elem for elem in text.split()])

# df['text'] = [remove_stopwords(elem) for elem in df['text']]

# # Remove words with a length of one or two
# def remove_short_words(text):
#     return ' '.join(['' if (len(elem) <= 2) else elem for elem in text.split()])

# df['text'] = [remove_short_words(elem) for elem in df['text']]


In [8]:
def preprocessing(text):
    text = "".join([x for x in text if x not in string.punctuation]) # Remmove punctuations 
    text = "".join([x.lower() for x in text]) # Convert to lower case
    text = ' '.join(['' if (x in all_stopwords) else x for x in text.split()]) # Remove stopwords
    text = re.sub("(\W|\d+|\n)", " ", text).strip() # remove spaces, digits and line breaks
    text = ' '.join(['' if (len(x) <= 2) else x for x in text.split()]) # Remove short words
    return(text)

In [9]:
df['text'] = [preprocessing(x) for x in df['text']]

In [10]:
# Define a function to evaluate the model performance in terms of F1 score
def evaluate(model, X, y):
    pred = model.predict(X)
    pred = [1 if p>=0.5 else 0 for p in pred]
    acc = np.sum(y == pred)/len(y)
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()

    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = (2*precision*recall)/(precision + recall)
    
    y_pred = model.predict(X)
    auc_roc = round(roc_auc_score(y, y_pred),4)
    pre, rec, thresholds = precision_recall_curve(y, y_pred)
    auc_pr = round(auc(rec, pre),4)
    return(auc_roc, auc_pr, acc, precision, recall, specificity, f1)

In [11]:
# Create the CNN model
def cnnmodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(Conv1D(8, kernel_size=8, activation="relu"))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())  
    model.add(Dense(units=1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [12]:
# Create the RNN model
def rnnmodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):    
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(layers.SimpleRNN(units=8)) 
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [13]:
# Create the GRU model
def grumodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):    
    model = Sequential()   
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(layers.GRU(units=8)) 
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [14]:
# Create the LSTM model
def lstmmodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):    
    model = Sequential() 
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(layers.LSTM(units=8)) 
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [15]:
# Create the LSTM model
def bi_lstm_model(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):    
    model = Sequential() 
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(layers.Bidirectional(layers.LSTM(8)))
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [17]:
# !pip install gensim
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('word_embeddings/BioWordVec_PubMed_MIMICIII_d200.vec.bin', binary=True)

In [19]:
# Create an embedding matrix for each word in the training set
def emb_matrix(word_index):
    count = 0
    words_not_found = []
    embedding_matrix = np.zeros((len(word_index) + 1, 200))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = model[word]
        except KeyError:
            count += 1
            words_not_found.append(word)
    return embedding_matrix,words_not_found,count

In [19]:
# Set the number of epochs and the batch size
epochs = 20 
batch_size = 32
max_length = 557
embedding_dim = 200
dropout_rate=0.3

In [20]:
# Run the models
X = df['text']
models = [cnnmodel,rnnmodel,grumodel,lstmmodel,bi_lstm_model]
model_names = ['cnnmodel','rnnmodel','grumodel','lstmmodel','bi_lstm_model']

time_1 = time.time()

for i in range(7): # Go through each disease type

    event_categorical = df['This_Disease_'+str(i)]
    dis_type = disease_types[i]
    
    time_start = time.time()   

    # Split the data 10 times into train and test sets with stratification
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25,random_state=0)
    
    j = 0
    for train_index, test_index in sss.split(X, event_categorical):   
              
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = event_categorical[train_index], event_categorical[test_index]
        
        tokenizer = Tokenizer(num_words=5000) # Tokenize the text
        tokenizer.fit_on_texts(x_train)
        x_train = tokenizer.texts_to_sequences(x_train)
        x_test = tokenizer.texts_to_sequences(x_test)
        word_index = tokenizer.word_index
        vocab_size = len(word_index) + 1 # plus the reserved index 0
 
        # Pad the sequences with 0's
        x_train = pad_sequences(x_train, padding='post', maxlen=max_length) 
        x_test = pad_sequences(x_test, padding='post', maxlen=max_length)
        
        embedding_matrix = emb_matrix(word_index)

        # Fit the models
        j += 1
        iteration = "iter" + str(j)
        
        for m in range(len(models)):    
            time_s = time.time()  
            mymodel = models[m]  
            this_model = mymodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix)
            this_model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
            time_e = time.time() - time_s

            # Collect and log evaluation metrics
            auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(this_model, x_test, y_test)

            with open('other/CNN_Paper_BioWordVec_20epocs.csv','a') as fd:
                fd.write(f'{dis_type},{iteration},{model_names[m]},{auc_roc},{auc_pr},{acc},{precision},{recall},{specificity},{f1},{time_e},{vocab_size},{len(x_train)}\n')
    
    # Average metrics
    running_time = time.time() - time_start
#     logging.info(f',{disease_types[i]}, running time, {running_time}')
    with open('other/CNN_Paper_BioWordVec_20epocs.csv','a') as fd:
        fd.write(f'{dis_type}, 10 iteration running time, {running_time}\n')

Epoch 1/20
29/29 [==============================] - 6s 68ms/step - loss: 0.2676 - auc: 0.5300
Epoch 2/20
29/29 [==============================] - 2s 68ms/step - loss: 0.2079 - auc: 0.6454
Epoch 3/20
29/29 [==============================] - 2s 66ms/step - loss: 0.1949 - auc: 0.7002
Epoch 4/20
29/29 [==============================] - 2s 77ms/step - loss: 0.1598 - auc: 0.8302
Epoch 5/20
29/29 [==============================] - 2s 86ms/step - loss: 0.1443 - auc: 0.9076
Epoch 6/20
29/29 [==============================] - 3s 89ms/step - loss: 0.1207 - auc: 0.9354
Epoch 7/20
29/29 [==============================] - 2s 83ms/step - loss: 0.0952 - auc: 0.9640
Epoch 8/20
29/29 [==============================] - 2s 81ms/step - loss: 0.0782 - auc: 0.9789
Epoch 9/20
29/29 [==============================] - 3s 89ms/step - loss: 0.0723 - auc: 0.9901
Epoch 10/20
29/29 [==============================] - 3s 88ms/step - loss: 0.0620 - auc: 0.9934
Epoch 11/20
29/29 [==============================] - 2s 80m

Epoch 7/20
29/29 [==============================] - 9s 315ms/step - loss: 0.2180 - auc: 0.5208
Epoch 8/20
29/29 [==============================] - 8s 262ms/step - loss: 0.2122 - auc: 0.5847
Epoch 9/20
29/29 [==============================] - 7s 254ms/step - loss: 0.2196 - auc: 0.5265
Epoch 10/20
29/29 [==============================] - 7s 246ms/step - loss: 0.2074 - auc: 0.6089
Epoch 11/20
29/29 [==============================] - 8s 262ms/step - loss: 0.2035 - auc: 0.6609
Epoch 12/20
29/29 [==============================] - 7s 252ms/step - loss: 0.2072 - auc: 0.6077
Epoch 13/20
29/29 [==============================] - 7s 240ms/step - loss: 0.2028 - auc: 0.6456
Epoch 14/20
29/29 [==============================] - 8s 263ms/step - loss: 0.1996 - auc: 0.6601
Epoch 15/20
29/29 [==============================] - 8s 262ms/step - loss: 0.1888 - auc: 0.7346
Epoch 16/20
29/29 [==============================] - 8s 277ms/step - loss: 0.1852 - auc: 0.7427
Epoch 17/20
29/29 [========================

29/29 [==============================] - 6s 221ms/step - loss: 0.2038 - auc: 0.6239
Epoch 18/20
29/29 [==============================] - 6s 207ms/step - loss: 0.1997 - auc: 0.6472
Epoch 19/20
29/29 [==============================] - 6s 206ms/step - loss: 0.1889 - auc: 0.6997
Epoch 20/20
29/29 [==============================] - 6s 222ms/step - loss: 0.1937 - auc: 0.6702
Epoch 1/20
29/29 [==============================] - 8s 190ms/step - loss: 0.5621 - auc: 0.4320
Epoch 2/20
29/29 [==============================] - 6s 203ms/step - loss: 0.4395 - auc: 0.4236
Epoch 3/20
29/29 [==============================] - 5s 190ms/step - loss: 0.2840 - auc: 0.4917
Epoch 4/20
29/29 [==============================] - 5s 188ms/step - loss: 0.2401 - auc: 0.5135
Epoch 5/20
29/29 [==============================] - 6s 210ms/step - loss: 0.2327 - auc: 0.4927
Epoch 6/20
29/29 [==============================] - 6s 190ms/step - loss: 0.2309 - auc: 0.4899
Epoch 7/20
29/29 [==============================] - 6s 196

29/29 [==============================] - 6s 217ms/step - loss: 0.4353 - auc: 0.4031
Epoch 4/20
29/29 [==============================] - 6s 204ms/step - loss: 0.3371 - auc: 0.4089
Epoch 5/20
29/29 [==============================] - 6s 205ms/step - loss: 0.2541 - auc: 0.4519
Epoch 6/20
29/29 [==============================] - 6s 217ms/step - loss: 0.2459 - auc: 0.3945
Epoch 7/20
29/29 [==============================] - 6s 205ms/step - loss: 0.2341 - auc: 0.4382
Epoch 8/20
29/29 [==============================] - 6s 208ms/step - loss: 0.2282 - auc: 0.4814
Epoch 9/20
29/29 [==============================] - 6s 217ms/step - loss: 0.2133 - auc: 0.5696
Epoch 10/20
29/29 [==============================] - 6s 205ms/step - loss: 0.2224 - auc: 0.5256
Epoch 11/20
29/29 [==============================] - 6s 218ms/step - loss: 0.2229 - auc: 0.5324
Epoch 12/20
29/29 [==============================] - 6s 206ms/step - loss: 0.2159 - auc: 0.5475
Epoch 13/20
29/29 [==============================] - 6s 20

29/29 [==============================] - 2s 82ms/step - loss: 0.0127 - auc: 1.0000
Epoch 14/20
29/29 [==============================] - 2s 84ms/step - loss: 0.0101 - auc: 1.0000
Epoch 15/20
29/29 [==============================] - 2s 81ms/step - loss: 0.0083 - auc: 1.0000
Epoch 16/20
29/29 [==============================] - 2s 80ms/step - loss: 0.0084 - auc: 1.0000
Epoch 17/20
29/29 [==============================] - 3s 92ms/step - loss: 0.0076 - auc: 1.0000
Epoch 18/20
29/29 [==============================] - 3s 101ms/step - loss: 0.0057 - auc: 1.0000
Epoch 19/20
29/29 [==============================] - 2s 78ms/step - loss: 0.0052 - auc: 1.0000
Epoch 20/20
29/29 [==============================] - 2s 78ms/step - loss: 0.0055 - auc: 1.0000
Epoch 1/20
29/29 [==============================] - 4s 106ms/step - loss: 0.5778 - auc: 0.4646
Epoch 2/20
29/29 [==============================] - 3s 117ms/step - loss: 0.4909 - auc: 0.4066
Epoch 3/20
29/29 [==============================] - 3s 113ms/

...

29/29 [==============================] - 3s 117ms/step - loss: 0.0362 - auc: 1.0000
Epoch 17/20
29/29 [==============================] - 3s 118ms/step - loss: 0.0295 - auc: 0.9999
Epoch 18/20
29/29 [==============================] - 3s 119ms/step - loss: 0.0266 - auc: 1.0000
Epoch 19/20
29/29 [==============================] - 3s 118ms/step - loss: 0.0241 - auc: 0.9999
Epoch 20/20
29/29 [==============================] - 3s 117ms/step - loss: 0.0198 - auc: 1.0000
Epoch 1/20
29/29 [==============================] - 4s 113ms/step - loss: 0.6090 - auc: 0.4680
Epoch 2/20
29/29 [==============================] - 3s 109ms/step - loss: 0.5311 - auc: 0.4320
Epoch 3/20
29/29 [==============================] - 3s 106ms/step - loss: 0.5054 - auc: 0.4666
Epoch 4/20
29/29 [==============================] - 3s 105ms/step - loss: 0.4790 - auc: 0.4920
Epoch 5/20
29/29 [==============================] - 3s 106ms/step - loss: 0.4607 - auc: 0.5004
Epoch 6/20
29/29 [==============================] - 3s 11

29/29 [==============================] - 8s 283ms/step - loss: 0.4125 - auc: 0.6220
Epoch 7/20
29/29 [==============================] - 8s 288ms/step - loss: 0.4001 - auc: 0.6704
Epoch 8/20
29/29 [==============================] - 8s 280ms/step - loss: 0.3919 - auc: 0.6838
Epoch 9/20
29/29 [==============================] - 8s 286ms/step - loss: 0.3883 - auc: 0.6810
Epoch 10/20
29/29 [==============================] - 8s 288ms/step - loss: 0.3754 - auc: 0.7296
Epoch 11/20
29/29 [==============================] - 8s 282ms/step - loss: 0.3621 - auc: 0.7626
Epoch 12/20
29/29 [==============================] - 8s 291ms/step - loss: 0.3531 - auc: 0.7845
Epoch 13/20
29/29 [==============================] - 8s 288ms/step - loss: 0.3431 - auc: 0.8029
Epoch 14/20
29/29 [==============================] - 8s 290ms/step - loss: 0.3183 - auc: 0.8346
Epoch 15/20
29/29 [==============================] - 8s 278ms/step - loss: 0.3092 - auc: 0.8398
Epoch 16/20
29/29 [==============================] - 8s

29/29 [==============================] - 6s 220ms/step - loss: 0.4027 - auc: 0.6335
Epoch 17/20
29/29 [==============================] - 6s 220ms/step - loss: 0.3863 - auc: 0.6764
Epoch 18/20
29/29 [==============================] - 6s 213ms/step - loss: 0.3976 - auc: 0.6205
Epoch 19/20
29/29 [==============================] - 7s 226ms/step - loss: 0.3856 - auc: 0.6677
Epoch 20/20
29/29 [==============================] - 6s 211ms/step - loss: 0.3781 - auc: 0.6776
Epoch 1/20
29/29 [==============================] - 9s 207ms/step - loss: 0.6159 - auc: 0.4664
Epoch 2/20
29/29 [==============================] - 6s 199ms/step - loss: 0.5452 - auc: 0.4452 4s - los
Epoch 3/20
29/29 [==============================] - 6s 201ms/step - loss: 0.4568 - auc: 0.4936
Epoch 4/20
29/29 [==============================] - 6s 204ms/step - loss: 0.4349 - auc: 0.5456
Epoch 5/20
29/29 [==============================] - 6s 206ms/step - loss: 0.4355 - auc: 0.5348
Epoch 6/20
29/29 [==============================

29/29 [==============================] - 3s 113ms/step - loss: 0.4261 - auc: 0.5920
Epoch 7/20
29/29 [==============================] - 3s 106ms/step - loss: 0.4279 - auc: 0.5872
Epoch 8/20
29/29 [==============================] - 3s 108ms/step - loss: 0.4334 - auc: 0.5691
Epoch 9/20
29/29 [==============================] - 3s 106ms/step - loss: 0.4292 - auc: 0.5729
Epoch 10/20
29/29 [==============================] - 3s 105ms/step - loss: 0.4322 - auc: 0.5630
Epoch 11/20
29/29 [==============================] - 3s 114ms/step - loss: 0.4324 - auc: 0.5635
Epoch 12/20
29/29 [==============================] - 3s 110ms/step - loss: 0.4282 - auc: 0.5679
Epoch 13/20
29/29 [==============================] - 3s 105ms/step - loss: 0.4071 - auc: 0.6476
Epoch 14/20
29/29 [==============================] - 3s 105ms/step - loss: 0.4237 - auc: 0.5880
Epoch 15/20
29/29 [==============================] - 3s 105ms/step - loss: 0.4222 - auc: 0.5848
Epoch 16/20
29/29 [==============================] - 4s

29/29 [==============================] - 8s 289ms/step - loss: 0.3491 - auc: 0.7843
Epoch 17/20
29/29 [==============================] - 8s 282ms/step - loss: 0.3462 - auc: 0.7785
Epoch 18/20
29/29 [==============================] - 8s 290ms/step - loss: 0.3342 - auc: 0.8145
Epoch 19/20
29/29 [==============================] - 8s 284ms/step - loss: 0.3214 - auc: 0.8307
Epoch 20/20
29/29 [==============================] - 8s 289ms/step - loss: 0.3089 - auc: 0.8491
Epoch 1/20
29/29 [==============================] - 4s 131ms/step - loss: 0.4572 - auc: 0.5159
Epoch 2/20
29/29 [==============================] - 4s 131ms/step - loss: 0.3974 - auc: 0.6790
Epoch 3/20
29/29 [==============================] - 4s 132ms/step - loss: 0.3398 - auc: 0.8404
Epoch 4/20
29/29 [==============================] - 4s 128ms/step - loss: 0.2975 - auc: 0.8874
Epoch 5/20
29/29 [==============================] - 4s 129ms/step - loss: 0.2598 - auc: 0.9239
Epoch 6/20
29/29 [==============================] - 4s 12

In [21]:
time_2 = time.time() - time_1  

In [22]:
time_2
with open('other/CNN_Paper_BioWordVec_20epocs.csv','a') as fd:
    fd.write(f'{dis_type}, Total running time, {time_2}\n')